In [ ]:
import os
import gym
import numpy as np
import cv2
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.utils import set_random_seed

# ==========================================
# 1. CONFIGURAZIONE CARTELLE E PERCORSI
# ==========================================
MODEL_PATH = "./train/nmod/mario_WINNER_STRIKE.zip" # Inserisci qui il tuo modello
TOTAL_TEST_RUNS = 15000
SEED = 42

class MarioModernized(gym.Env):
    def __init__(self):
        super().__init__()
        # Carica il gioco originale
        inner_env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
        self.mario = JoypadSpace(inner_env, SIMPLE_MOVEMENT)
        
        # IA vede 84x84 in scala di grigi
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = self.mario.action_space
        
        # Variabili di stato interne
        self.max_x = 0
        self.stagnant_steps = 0
        self.skip_frames = 4 
        self.view_stack = [np.zeros((84, 84), dtype=np.uint8)] * 4

    def _process_frame(self, frame):
        if frame is not None:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            
            # Visualizzazione Memoria Temporale (Film Strip)
            self.view_stack.pop(0)
            self.view_stack.append(resized)
            film_strip = np.hstack(self.view_stack)
            big_strip = cv2.resize(film_strip, (84*4*2, 84*2), interpolation=cv2.INTER_NEAREST)
            cv2.imshow("Visione IA (Memoria 4 Frame)", big_strip)
            cv2.waitKey(1)
            
            return resized[:, :, np.newaxis]
        return np.zeros((84, 84, 1), dtype=np.uint8)

    def reset(self):
        self.max_x = 0
        self.stagnant_steps = 0
        self.view_stack = [np.zeros((84, 84), dtype=np.uint8)] * 4
        obs = self.mario.reset()
        return self._process_frame(obs)

    def step(self, action):
        for _ in range(self.skip_frames):
            obs, reward, done, info = self.mario.step(action)
            if done: break

        self.mario.render()
        
        # Logica per rilevare lo stagnamento
        x_pos = info['x_pos']
        if x_pos > self.max_x:
            self.max_x = x_pos
            self.stagnant_steps = 0
        else:
            self.stagnant_steps += 1
        
        # Se Mario è bloccato per più di 100 decisioni
        if self.stagnant_steps > 100:
            info['stuck'] = True
            done = True
        
        obs = self._process_frame(obs)
        return obs, reward, done, info

# ==========================================
# 2. CICLO DI VALUTAZIONE (100 RUN)
# ==========================================
if __name__ == "__main__":
    # Settaggio seed per la riproducibilità
    set_random_seed(SEED)
    
    # Inizializzazione Ambiente
    env = DummyVecEnv([lambda: MarioModernized()])
    env = VecFrameStack(env, n_stack=4, channels_order='last')

    if not os.path.exists(MODEL_PATH):
        print(f"ERRORE: Modello non trovato in {MODEL_PATH}")
        exit()

    # Caricamento Modello
    print(f"Caricamento modello in corso...")
    model = PPO.load(MODEL_PATH, env=env)

    # Dizionario per accumulare le statistiche
    results = {
        "vittorie": 0,
        "morte_nemico": 0,
        "morte_fosso": 0,
        "morte_tempo_blocco": 0,
        "punteggi_gioco": [],  # Score Nintendo
        "tempi_completamento": []
    }

    print(f"Inizio test su {TOTAL_TEST_RUNS} partite consecutive...")

    for i in range(1, TOTAL_TEST_RUNS + 1):
        obs = env.reset()
        done = False
        start_time_game = None
        
        while not done:
            action, _ = model.predict(obs, deterministic=False)
            obs, reward, done, info = env.step(action)
            
            # Estrazione dati dall'ambiente vettorializzato
            current_info = info[0]
            
            if start_time_game is None:
                start_time_game = current_info['time']

            if done:
                # Salvataggio Punteggio Nintendo (Score)
                results["punteggi_gioco"].append(current_info['score'])
                
                # Caso: Vittoria (Tocca la bandiera)
                if current_info.get('flag_get'):
                    results["vittorie"] += 1
                    tempo_impiegato = start_time_game - current_info['time']
                    results["tempi_completamento"].append(tempo_impiegato)
                    print(f"Partita {i:03d}: VITTORIA | Score: {current_info['score']} | Tempo: {tempo_impiegato}")
                
                # Caso: Fallimento
                else:
                    if current_info.get('stuck') or current_info['time'] <= 0:
                        results["morte_tempo_blocco"] += 1
                        causa = "BLOCCATO/TIME"
                    elif current_info['y_pos'] < 70: # Altezza caduta nel vuoto
                        results["morte_fosso"] += 1
                        causa = "CADUTA FOSSO"
                    else:
                        results["morte_nemico"] += 1
                        causa = "NEMICO/OSTAC"
                    print(f"Partita {i:03d}: {causa:12s} | Score: {current_info['score']}")

    # ==========================================
    # 3. REPORT FINALE DELLE PERFORMANCE
    # ==========================================
    print("\n" + "="*50)
    print("                VALUTAZIONE IA FINALE")
    print("="*50)
    print(f"Punteggio Nintendo Medio:  {np.mean(results['punteggi_gioco']):.2f}")
    print(f"Punteggio Nintendo Max:    {np.max(results['punteggi_gioco'])}")
    print("-" * 50)
    print(f"TASSO DI SUCCESSO (Win):   {results['vittorie']}%")
    print(f"TASSO DI FALLIMENTO:       {100 - results['vittorie']}%")
    print("-" * 50)
    print("ANALISI DELLE MORTI:")
    print(f"  - Per nemici o ostacoli: {results['morte_nemico']}%")
    print(f"  - Per caduta nei fossi:  {results['morte_fosso']}%")
    print(f"  - Per tempo o blocchi:   {results['morte_tempo_blocco']}%")
    print("-" * 50)
    
    if results["vittorie"] > 0:
        media_tempo = np.mean(results['tempi_completamento'])
        print(f"Tempo medio di vittoria:   {media_tempo:.2f} unità di gioco")
    else:
        print("Tempo medio di vittoria:   N/A (Nessuna vittoria)")
    print("="*50)

    cv2.destroyAllWindows()

Caricamento modello in corso...
Wrapping the env in a VecTransposeImage.
Inizio test su 15000 partite consecutive...


C:\Users\matte\Desktop\marioia\mario_stabile\Lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Partita 001: NEMICO/OSTAC | Score: 200
Partita 002: VITTORIA | Score: 200 | Tempo: 74
Partita 003: VITTORIA | Score: 500 | Tempo: 64
Partita 004: NEMICO/OSTAC | Score: 500
